# 实验目的

# 编写生成斐波那契数列的函数并调用

In [11]:
def fib(n):
    a, b = 1, 1
    while a < n:
        print(a, end=' ')
        a, b = b, a+b
    print()

fib(1000)

1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 


# 函数嵌套定义

In [12]:
def myMap(iterable, op, value):     #自定义函数
    if op not in '+-*/':
        return 'Error operator'
    def nested(item):                   #嵌套定义函数
        return eval(repr(item)+op+repr(value))
    return map(nested, iterable)        #使用在函数内部定义的函数
list(myMap(range(5), '+', 5))       #调用外部函数，不需要关心其内部实现

[5, 6, 7, 8, 9]

In [19]:
list(myMap(range(5), '-', 5))

[-5, -4, -3, -2, -1]

# 用函数嵌套定义和递归实现帕斯卡公式C(n,i) = C(n-1, i) + C(n-1, i-1)，进行组合数C(n,i)的快速求解。


In [25]:
def f2(n,i):
    cache2 = dict()
    
    def f(n,i):
        if n==i or i==0:
            return 1
        elif (n,i) not in cache2:
            cache2[(n,i)] = f(n-1, i) + f(n-1, i-1)
        return cache2[(n,i)]
    
    return f(n,i)


In [20]:
from functools import lru_cache

@lru_cache(maxsize=64)
def cni(n, i):
    if n==i or i==0:
        return 1
    return cni(n-1, i) + cni(n-1, i-1)

In [24]:
def linear(a, b):
    def result(x):              #在Python中，函数是可以嵌套定义的
        return a * x + b
    return result               #返回可被调用的函数


In [26]:

class linear:
    def __init__(self, a, b):
        self.a, self.b = a, b
    def __call__(self, x):         #这里是关键
        return self.a * x + self.b


In [27]:
def before(func):                       #定义修饰器
    def wrapper(*args, **kwargs):
        print('Before function called.')
        return func(*args, **kwargs)
    return wrapper

def after(func):                        #定义修饰器
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print('After function called.')
        return result
    return wrapper

@before
@after
def test():                             #同时使用两个修饰器改造函数
    print(3)
#调用被修饰的函数
test()


Before function called.
3
After function called.


# 使用递归法对整数进行因数分解。

In [28]:
from random import randint

def factors(num, fac=[]):
    #每次都从2开始查找因数
    for i in range(2, int(num**0.5)+1):
        #找到一个因数
        if num%i == 0:
            fac.append(i)
            #对商继续分解，重复这个过程
            factors(num//i, fac)
            #注意，这个break非常重要
            break
    else:
        #不可分解了，自身也是个因数
        fac.append(num)
facs = []
n = randint(2, 10**8)
factors(n, facs)
result = '*'.join(map(str, facs))
if n==eval(result):
    print(n, '= '+result)


64172922 = 2*3*11*653*1489


# 对于绝大多数情况下，在函数内部直接修改形参的值不会影响实参，而是创建一个新变量

In [30]:
def addOne(a):
    print(id(a), ':', a)
    a += 1
    print(id(a), ':', a)
v = 3
id(v)


1351541456

In [31]:
addOne(v)

1351541456 : 3
1351541472 : 4


In [32]:
v

3

In [33]:
id(v)

1351541456

# 可以通过特殊的方式在函数内部修改实参的值。

In [35]:
def modify(v):          # 使用下标修改列表元素值
    v[0] = v[0]+1
a = [2]
modify(a)
a


[3]

In [36]:
def modify(v, item):    # 使用列表的方法为列表增加元素
    v.append(item)
a = [2]
modify(a,3)
a


[2, 3]

# 如果传递给函数的实参是可变序列，并且在函数内部使用下标或可变序列自身的方法增加、删除元素或修改元素时，实参也得到相应的修改

In [37]:
def modify(d):         #修改字典元素值或为字典增加元素
    d['age'] = 38
a = {'name':'Dong', 'age':37, 'sex':'Male'}
a


{'name': 'Dong', 'age': 37, 'sex': 'Male'}

In [38]:
modify(a)
a


{'name': 'Dong', 'age': 38, 'sex': 'Male'}

# 调用函数时实参和形参的顺序必须严格一致，并且实参和形参的数量必须相同。


In [40]:
def demo(a, b, c):
    print(a, b, c)
demo(3, 4, 5)                   #按位置传递参数

3 4 5


In [41]:
demo(3, 5, 4)

3 5 4


In [42]:
demo(1, 2, 3, 4)                #实参与形参数量必须相同

TypeError: demo() takes 3 positional arguments but 4 were given

# 多次调用函数并且不为默认值参数传递值时，默认值参数只在定义时进行一次解释和初始化，对于列表、字典这样可变类型的默认值参数，这一点可能会导致很严重的逻辑错误

In [56]:
def demo(newitem, old_list=[]):
    old_list.append(newitem)
    return old_list

In [57]:
print(demo('5', [1, 2, 3, 4]))


[1, 2, 3, 4, '5']


In [58]:
print(demo('aaa', ['a', 'b']))


['a', 'b', 'aaa']


In [59]:
print(demo('a'))


['a']


In [60]:
print(demo('b'))                 #注意这里的输出结果

['a', 'b']


# 函数的默认值参数是在函数定义时确定值的，所以只会被初始化一次。


In [63]:
i = 3
def f(n=i):                 #参数n的值仅取决于i的当前值
    print(n)
f()


3


In [64]:
i = 5                       #函数定义后修改i的值不影响参数n的默认值
f()


3


In [65]:
i = 7
f()


3


In [66]:
def f(n=i):                 #重新定义函数
    print(n)
f()


7


# *parameter的用法

In [67]:
def demo(*p):
    print(p)

In [68]:
demo(1,2,3)

(1, 2, 3)


In [69]:
demo(1,2)

(1, 2)


In [70]:
demo(1,2,3,4,5,6,7)

(1, 2, 3, 4, 5, 6, 7)


# **parameter的用法

In [71]:
def demo(**p):
    for item in p.items():
        print(item)


In [72]:
demo(x=1,y=2,z=3)

('x', 1)
('y', 2)
('z', 3)


# 变量作用域

In [74]:
def demo():
    global x
    x = 3
    y = 4
    print(x,y)

In [75]:
x = 5
demo()

3 4


In [76]:
x

3

In [77]:
y

NameError: name 'y' is not defined

In [78]:
del x
x

NameError: name 'x' is not defined

In [79]:
demo()

3 4


In [80]:
x

3

In [81]:
y

NameError: name 'y' is not defined

# lambda表达式

In [82]:
f = lambda x, y, z: x+y+z        #可以给lambda表达式起名字
f(1,2,3)                         #像函数一样调用

6

In [83]:
g = lambda x, y=2, z=3: x+y+z    #参数默认值
g(1)

6

In [84]:
g(2, z=4, y=5)                   #关键参数

11

In [85]:
L = [(lambda x: x**2),
         (lambda x: x**3),
         (lambda x: x**4)]
print(L[0](2),L[1](2),L[2](2))


4 8 16


In [86]:
D = {'f1':(lambda:2+3),
         'f2':(lambda:2*3),         
         'f3':(lambda:2**3)}
print(D['f1'](), D['f2'](), D['f3']())

5 6 8


In [87]:
 L = [1,2,3,4,5]
 print(list(map(lambda x: x+10, L)))        #模拟向量运算

[11, 12, 13, 14, 15]


In [88]:
L

[1, 2, 3, 4, 5]

In [89]:
def demo(n):
    return n*n

demo(5)

25

In [92]:
a_list = [1,2,3,4,5]
list(map(lambda x: demo(x), a_list))  #在lambda表达式中调用函数

[1, 4, 9, 16, 25]

In [93]:
data = list(range(20))           #创建列表
data

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [94]:
import random
random.shuffle(data)             #打乱顺序
data

[15, 10, 8, 16, 18, 12, 1, 4, 11, 5, 9, 14, 19, 6, 7, 3, 13, 17, 2, 0]

In [95]:
data.sort(key=lambda x: x)       #和不指定规则效果一样
data

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [96]:
data.sort(key=lambda x: len(str(x)))     #按转换成字符串以后的长度排序
data

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [97]:
data.sort(key=lambda x: len(str(x)), reverse=True)#降序排序
data

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [98]:
import random
x = [[random.randint(1,10) for j in range(5)] for i in range(5)]
                                    #使用列表推导式创建列表
                                    #包含5个子列表的列表
                                    #每个子列表中包含5个1到10之间的随机数
for item in x:
    print(item)	


[8, 7, 2, 7, 8]
[2, 10, 1, 3, 3]
[10, 7, 6, 8, 6]
[3, 4, 9, 6, 4]
[10, 9, 9, 6, 3]


In [99]:
y = sorted(x, key=lambda item: (item[1], item[4]))
                               #按子列表中第2个元素升序、第5个元素升序排序
for item in y:
    print(item)

[3, 4, 9, 6, 4]
[10, 7, 6, 8, 6]
[8, 7, 2, 7, 8]
[10, 9, 9, 6, 3]
[2, 10, 1, 3, 3]


# 生成器函数设计要点

In [100]:
def f():
    a, b = 1, 1            #序列解包，同时为多个元素赋值
    while True:
        yield a            #暂停执行，需要时再产生一个新元素
        a, b = b, a+b      #序列解包，继续生成新元素
a = f()                #创建生成器对象
for i in range(10):    #斐波那契数列中前10个元素
    print(a.__next__(), end=' ')

1 1 2 3 5 8 13 21 34 55 

In [101]:
for i in f():         #斐波那契数列中第一个大于100的元素
    if i > 100:
        print(i, end=' ')
        break

144 

In [106]:
a = f()               #创建生成器对象
next(a)               #使用内置函数next()获取生成器对象中的元素

1

In [107]:
next(a)               #每次索取新元素时，由yield语句生成

1

In [108]:
a.__next__()          #也可以调用生成器对象的__next__()方法

2

In [109]:
a.__next__()

3

In [113]:
def f():
    yield from 'abcdefg'        #使用yield表达式创建生成器
	
x = f()
next(x)

'a'

In [114]:
next(x)

'b'

In [115]:
for item in x:              #输出x中的剩余元素
    print(item, end=' ')

c d e f g 

In [116]:
def gen():
    yield 1
    yield 2
    yield 3
	
x, y, z = gen()          #生成器对象支持序列解包


In [117]:
def count(start, step):
    num = start
    while True:            #无穷循环
        yield num          #返回一个数，暂停执行，等待下一次索要数据
        num += step
x = count(3, 5)
for i in range(10):
    print(next(x), end=' ')

3 8 13 18 23 28 33 38 43 48 

In [118]:
for i in range(10):
    print(next(x), end=' ')

53 58 63 68 73 78 83 88 93 98 

# 精彩案例赏析

编写函数，接收任意多个实数，返回一个元组，其中第一个元素为所有参数的平均值，其他元素为所有参数中大于平均值的实数。

In [120]:
def demo(*para):
    avg = sum(para) / len(para)            #平均值
    g = [i for i in para if i>avg]         #列表推导式
    return (avg,) + tuple(g)

编写函数，接收字符串参数，返回一个元组，其中第一个元素为大写字母个数，第二个元素为小写字母个数。

In [121]:
def demo(s):
    result = [0, 0]
    for ch in s:
        if ch.islower():
            result[1] += 1
        elif ch.isupper():
            result[0] += 1
    return tuple(result)


编写函数，接收包含n个整数的列表lst和一个整数k（0<=k<n）作为参数，返回新列表。处理规则为：将列表lst中下标k之前的元素逆序，下标k之后的元素逆序，然后将整个列表lst中的所有元素逆序。

In [122]:
def demo(lst, k):
    x = lst[k-1::-1]
    y = lst[:k-1:-1]
    return list(reversed(x+y))


本例描述的实际上是将列表循环左移k位的算法实现，下面的代码使用了更加直接的方法，但对于长列表来说效率远不如上面的代码高，因为pop(0)操作在列表首部删除元素，这会引起大量元素的前移。


In [123]:
def demo(lst, k):
    temp = lst[:]
    for i in range(k):
        temp.append(temp.pop(0))
    return temp


编写函数，接收一个整数t为参数，打印杨辉三角前t行。


In [125]:
def yanghui(t):
    print([1])
    line = [1, 1]
    print(line)
    for i in range(2, t):
        r = []
        for j in range(0, len(line)-1):
            r.append(line[j]+line[j+1])
        line = [1]+r+[1]
        print(line)


编写函数，使用collections标准库的defaultdict实现上例的功能

In [126]:
from collections import defaultdict

def yanghui(n):
    #所有元素默认值0
    triangle = defaultdict(int)
    for row in range(n):
        #每行第一个元素为1
        triangle[row,0] = 1
        print(triangle[row,0], end='\t')
        #生成该行后续元素
        for col in range(1, row+1):
            #如果指定位置的元素不存在，默认为0
            triangle[row,col] = triangle[row-1,col-1] + triangle[row-1,col]
            print(triangle[row,col], end='\t')
        print()

yanghui(14)


1	
1	1	
1	2	1	
1	3	3	1	
1	4	6	4	1	
1	5	10	10	5	1	
1	6	15	20	15	6	1	
1	7	21	35	35	21	7	1	
1	8	28	56	70	56	28	8	1	
1	9	36	84	126	126	84	36	9	1	
1	10	45	120	210	252	210	120	45	10	1	
1	11	55	165	330	462	462	330	165	55	11	1	
1	12	66	220	495	792	924	792	495	220	66	12	1	
1	13	78	286	715	1287	1716	1716	1287	715	286	78	13	1	


编写函数，接收一个正偶数为参数，输出两个素数，并且这两个素数之和等于原来的正偶数。如果存在多组符合条件的素数，则全部输出。


In [128]:
def demo(n):
    def IsPrime(p):
        if p == 2:
            return True
        if p%2 == 0:
            return False
        for i in range(3, int(p**0.5)+1, 2):
            if p%i==0:
                return False
        return True
    
    if isinstance(n, int) and n>0 and n%2==0:
        for i in range(2, n//2+1):
            if IsPrime(i) and IsPrime(n-i):
                print(i, '+', n-i, '=', n)


编写函数，接收两个正整数作为参数，返回一个元组，其中第一个元素为最大公约数，第二个元素为最小公倍数。


In [129]:
def demo(m, n):
    p = m*n
    while m%n!=0:
        m, n = n, m%n
    return (n, p//n)


编写函数，接收一个所有元素值都不相等的整数列表x和一个整数n，要求将值为n的元素作为支点，将列表中所有值小于n的元素全部放到n的前面，所有值大于n的元素放到n的后面。


In [130]:
def demo(x, n):
    t1 = [i for i in x if i<n]
    t2 = [i for i in x if i>n]
    return t1 + [n] + t2


编写函数，计算字符串匹配的准确率。
以打字练习程序为例，假设origin为原始内容，userInput为用户输入的内容，下面的代码用来测试用户输入的准确率。

In [131]:
def Rate(origin, userInput):
    if not (isinstance(origin, str) and isinstance(userInput, str)):
        print('The two parameters must be strings.')
        return
    right = sum((1 for o, u in zip(origin, userInput) if o==u))
    return round(right/len(origin), 2)


编写函数，使用非递归方法对整数进行因数分解。

In [133]:
from random import randint
from math import sqrt

def factoring(n):
    '''对大数进行因数分解'''
    if not isinstance(n, int):
        print('You must give me an integer')
        return
    #开始分解，把所有因数都添加到result列表中
    result = []
    for p in primes:
        while n!=1:
            if n%p == 0:
                n = n/p
                result.append(p)
            else:
                break
        else:
            result = '*'.join(map(str, result))
            return result
    #考虑参数本身就是素数的情况
    if not result:
        return n
testData = [randint(10, 100000) for i in range(50)]
#随机数中的最大数
maxData = max(testData)
#小于maxData的所有素数
primes = [ p for p in range(2, maxData) if 0 not in 
          [p%d for d in range(2, int(sqrt(p))+1)]]

for data in testData:
    r = factoring(data)
    print(data, '=', r)
    #测试分解结果是否正确
    print(data==eval(r))


72124 = 2*2*13*19*73
True
86885 = 5*17377
True
22167 = 3*3*3*821
True
42746 = 2*11*29*67
True
72232 = 2*2*2*9029
True
69293 = 7*19*521
True
69704 = 2*2*2*8713
True
38719 = 31*1249
True
96960 = 2*2*2*2*2*2*3*5*101
True
93025 = 5*5*61*61
True
47351 = 47351
True
91568 = 2*2*2*2*59*97
True
58229 = 58229
True
76431 = 3*73*349
True
40551 = 3*7*1931
True
68184 = 2*2*2*3*3*947
True
14718 = 2*3*11*223
True
72580 = 2*2*5*19*191
True
71501 = 127*563
True
9013 = 9013
True
19837 = 83*239
True
17954 = 2*47*191
True
82016 = 2*2*2*2*2*11*233
True
50398 = 2*113*223
True
14988 = 2*2*3*1249
True
83236 = 2*2*20809
True
4039 = 7*577
True
7574 = 2*7*541
True
13715 = 5*13*211
True
51788 = 2*2*11*11*107
True
40689 = 3*3*3*11*137
True
40891 = 103*397
True
15274 = 2*7*1091
True
42335 = 5*8467
True
53784 = 2*2*2*3*3*3*3*83
True
53242 = 2*7*3803
True
52431 = 3*17477
True
59893 = 101*593
True
64746 = 2*3*3*3*11*109
True
30714 = 2*3*5119
True
64602 = 2*3*3*37*97
True
86051 = 7*19*647
True
89025 = 3*5*5*1187
True
56

编写函数模拟猜数游戏。系统随机产生一个数，玩家最多可以猜5次，系统会根据玩家的猜测进行提示，玩家则可以根据系统的提示对下一次的猜测进行适当调整。


In [134]:
from random import randint

def guess(maxValue=100, maxTimes=5):
    #随机生成一个整数
    value = randint(1,maxValue)
    for i in range(maxTimes):
        prompt = 'Start to GUESS:' if i==0 else 'Guess again:'
        #使用异常处理结构，防止输入不是数字的情况
        try:
            x = int(input(prompt))
        except:
            print('Must input an integer between 1 and ', maxValue)
        else:
            #猜对了
            if x == value:
                print('Congratulations!')
                break
            elif x > value:
                print('Too big')
            else:
                print('Too little')
    else:
        #次数用完还没猜对，游戏结束，提示正确答案
        print('Game over. FAIL.')
        print('The value is ', value)


编写函数，计算形式如a+aa+aaa+aaaa+...+aaa...aaa的表达式的值，其中a为小于10的自然数。

In [135]:
def demo(v, n):
    assert type(n)==int and 0<v<10, 'v must be integer between 1 and 9'
    result, t = 0, 0
    for i in range(n):
        t = t*10 + v
        result += t
    return result

print(demo(3, 4))


3702


编写函数模拟报数游戏。有n个人围成一圈，顺序编号，从第一个人开始从1到k（假设k=3）报数，报到k的人退出圈子，然后圈子缩小，从下一个人继续游戏，问最后留下的是原来的第几号。


In [136]:
from itertools import cycle

def demo(lst, k):
    #切片，以免影响原来的数据
    t_lst = lst[:]
    #游戏一直进行到只剩下最后一个人
    while len(t_lst)>1:
        #创建cycle对象
        c = cycle(t_lst)
        #从1到k报数
        for i in range(k):
            t = next(c)
        #一个人出局，圈子缩小
        index = t_lst.index(t)
        t_lst = t_lst[index+1:] + t_lst[:index]
    #游戏结束
    return t_lst[0]

lst = list(range(1,11))
print(demo(lst, 3))


4


编写函数计算任意位数的黑洞数。黑洞数是指这样的整数：由这个数字每位上的数字组成的最大数减去每位数字组成的最小数仍然得到这个数自身。例如3位黑洞数是495，因为954-459=495，4位数字是6174，因为7641-1467=6174。

In [139]:
def main(n):
    '''参数n表示数字的位数，例如n=3时返回495，n=4时返回6174'''
    #待测试数范围的起点和结束值
    start = 10**(n-1)
    end = 10**n
    #依次测试每个数
    for i in range(start, end):
        #由这几个数字组成的最大数和最小数
        big = ''.join(sorted(str(i),reverse=True))
        little = ''.join(reversed(big))
        big, little = map(int,(big, little))
        if big-little == i:
            print(i)
n = 4
main(n)


6174


24点游戏是指随机选取4张扑克牌（不包括大小王），然后通过四则运算来构造表达式，如果表达式的值恰好等于24就赢一次。下面的代码定义了一个函数用来测试随机给定的4个数是否符合24点游戏规则，如果符合就输出所有可能的表达式。


In [143]:
from random import randint
from itertools import permutations

#4个数字和2个运算符可能组成的表达式形式
exps = ('((%s %s %s) %s %s) %s %s',
        '(%s %s %s) %s (%s %s %s)', 
        '(%s %s (%s %s %s)) %s %s',
        '%s %s ((%s %s %s) %s %s)',
        '%s %s (%s %s (%s %s %s))')
ops = r'+-*/'
def test24(v):
    result = []
    #Python允许函数的嵌套定义
    #这个函数对字符串表达式求值并验证是否等于24
    def check(exp):
        try:
            #有可能会出现除0异常，所以放到异常处理结构中
            return int(eval(exp)) == 24
        except:
            return False
    #全排列，枚举4个数的所有可能顺序
    for a in permutations(v):
        #查找4个数的当前排列能实现24的表达式
        t = [exp % (a[0], op1, a[1], op2, a[2], op3, a[3]) for op1 in ops for op2 in ops for op3 in ops for exp in exps if check(exp %(a[0], op1, a[1], op2, a[2], op3, a[3]))]
        if t:
            result.append(t)
    return result
for i in range(20):
    print('='*20)
    #生成随机数字进行测试
    lst = [randint(1, 14) for j in range(4)]
    r = test24(lst)
    if r:
        print(r)
    else:
        print('No answer for ', lst)


[['(7 + (13 * 7)) / 4', '((7 * 13) + 7) / 4'], ['(7 + (7 * 13)) / 4'], ['7 * (4 - (7 / 13))'], ['((13 * 7) + 7) / 4'], ['((13 * 7) + 7) / 4'], ['(7 + (7 * 13)) / 4'], ['(7 + (13 * 7)) / 4', '((7 * 13) + 7) / 4'], ['7 * (4 - (7 / 13))'], ['(4 - (7 / 13)) * 7'], ['(4 - (7 / 13)) * 7']]
[['10 + ((4 - 3) * 14)', '(10 * (4 - 3)) + 14', '(10 / (4 - 3)) + 14'], ['(10 + 14) * (4 - 3)', '10 + (14 * (4 - 3))', '(10 + 14) / (4 - 3)', '10 + (14 / (4 - 3))'], ['(10 + 14) + (3 / 4)', '10 + (14 + (3 / 4))', '10 - (14 * (3 - 4))', '10 - (14 / (3 - 4))'], ['(10 + (3 / 4)) + 14', '10 + ((3 / 4) + 14)', '10 - ((3 - 4) * 14)', '((10 - 3) / 4) * 14', '(10 - 3) / (4 / 14)'], ['((10 - 3) * 14) / 4', '(10 - 3) * (14 / 4)'], ['((4 - 10) + 14) * 3', '(4 - (10 - 14)) * 3'], ['((4 + 14) - 10) * 3', '(4 + (14 - 10)) * 3'], ['((4 - 3) * 10) + 14', '(4 - 3) * (10 + 14)'], ['((4 - 3) * 14) + 10', '(4 - 3) * (14 + 10)'], ['(14 + 10) * (4 - 3)', '14 + (10 * (4 - 3))', '(14 + 10) / (4 - 3)', '14 + (10 / (4 - 3))', '((14

[['((13 - 1) + 3) + 9', '(13 - 1) + (3 + 9)', '(13 - (1 - 3)) + 9', '13 - (1 - (3 + 9))', '13 - ((1 - 3) - 9)'], ['((13 - 1) + 9) + 3', '(13 - 1) + (9 + 3)', '(13 - (1 - 9)) + 3', '13 - (1 - (9 + 3))', '13 - ((1 - 9) - 3)'], ['((13 + 3) - 1) + 9', '(13 + (3 - 1)) + 9', '13 + ((3 - 1) + 9)', '(13 + 3) - (1 - 9)', '13 + (3 - (1 - 9))'], ['((13 + 3) + 9) - 1', '(13 + 3) + (9 - 1)', '(13 + (3 + 9)) - 1', '13 + ((3 + 9) - 1)', '13 + (3 + (9 - 1))'], ['((13 + 9) - 1) + 3', '(13 + (9 - 1)) + 3', '13 + ((9 - 1) + 3)', '(13 + 9) - (1 - 3)', '13 + (9 - (1 - 3))'], ['((13 + 9) + 3) - 1', '(13 + 9) + (3 - 1)', '(13 + (9 + 3)) - 1', '13 + ((9 + 3) - 1)', '13 + (9 + (3 - 1))'], ['((3 + 13) - 1) + 9', '(3 + (13 - 1)) + 9', '3 + ((13 - 1) + 9)', '(3 + 13) - (1 - 9)', '3 + (13 - (1 - 9))'], ['((3 + 13) + 9) - 1', '(3 + 13) + (9 - 1)', '(3 + (13 + 9)) - 1', '3 + ((13 + 9) - 1)', '3 + (13 + (9 - 1))'], ['((3 - 1) + 13) + 9', '(3 - 1) + (13 + 9)', '(3 - (1 - 13)) + 9', '3 - (1 - (13 + 9))', '3 - ((1 - 13)

[['((12 + 13) - 13) * 2', '(12 + (13 - 13)) * 2', '((12 - 13) + 13) * 2', '(12 - (13 - 13)) * 2', '12 * ((13 - 13) + 2)', '12 * (13 - (13 - 2))', '((12 * 13) / 13) * 2', '(12 * (13 / 13)) * 2', '12 * ((13 / 13) * 2)', '(12 * 13) / (13 / 2)', '12 * (13 / (13 / 2))', '((12 / 13) * 13) * 2', '(12 / 13) * (13 * 2)', '(12 / (13 / 13)) * 2', '12 / (13 / (13 * 2))', '12 / ((13 / 13) / 2)'], ['(12 + 13) - (2 / 13)', '12 + (13 - (2 / 13))', '12 * ((13 + 2) - 13)', '12 * (13 + (2 - 13))', '((12 * 13) * 2) / 13', '(12 * 13) * (2 / 13)', '(12 * (13 * 2)) / 13', '12 * ((13 * 2) / 13)', '12 * (13 * (2 / 13))', '((12 / 13) * 2) * 13', '(12 / 13) * (2 * 13)', '(12 / (13 / 2)) * 13', '12 / (13 / (2 * 13))', '12 / ((13 / 2) / 13)'], ['((12 + 13) - 13) * 2', '(12 + (13 - 13)) * 2', '((12 - 13) + 13) * 2', '(12 - (13 - 13)) * 2', '12 * ((13 - 13) + 2)', '12 * (13 - (13 - 2))', '((12 * 13) / 13) * 2', '(12 * (13 / 13)) * 2', '12 * ((13 / 13) * 2)', '(12 * 13) / (13 / 2)', '12 * (13 / (13 / 2))', '((12 / 13

No answer for  [11, 5, 6, 7]
[['(5 - 3) * (13 - 1)'], ['(3 - 5) * (1 - 13)'], ['(3 * 1) * (13 - 5)', '3 * ((1 * 13) - 5)', '3 * (1 * (13 - 5))', '(3 / 1) * (13 - 5)', '3 / (1 / (13 - 5))'], ['(3 * (13 - 5)) * 1', '3 * ((13 - 5) * 1)', '3 * (13 - (5 * 1))', '(3 * (13 - 5)) / 1', '3 * ((13 - 5) / 1)', '3 * (13 - (5 / 1))'], ['3 * (13 - (1 * 5))', '3 * ((13 * 1) - 5)', '3 * ((13 / 1) - 5)'], ['(1 * 3) * (13 - 5)', '1 * (3 * (13 - 5))'], ['((1 * 13) - 5) * 3', '(1 * (13 - 5)) * 3', '1 * ((13 - 5) * 3)'], ['(1 - 13) * (3 - 5)'], ['((13 - 5) * 3) * 1', '(13 - 5) * (3 * 1)', '((13 - 5) * 3) / 1', '(13 - 5) * (3 / 1)'], ['((13 - 5) * 1) * 3', '(13 - 5) * (1 * 3)', '(13 - (5 * 1)) * 3', '((13 - 5) / 1) * 3', '(13 - (5 / 1)) * 3', '(13 - 5) / (1 / 3)'], ['(13 - 1) * (5 - 3)', '(13 - (1 * 5)) * 3', '((13 * 1) - 5) * 3', '((13 / 1) - 5) * 3']]
[['(4 * 6) + (1 / 5)', '4 * (6 + (1 / 5))'], ['4 * ((1 / 5) + 6)', '4 / (1 - (5 / 6))'], ['(6 * 4) + (1 / 5)'], ['6 / ((5 / 4) - 1)'], ['(6 + (1 / 5)) * 4']

编写函数，查找序列元素的最大值和最小值。给定一个序列，返回一个元组，其中元组第一个元素为序列最大值，第二个元素为序列最小值。

In [144]:
def myMaxMin(iterable):
    '''返回序列的最大值和最小值'''
    tMax = tMin = iterable[0]
    for item in iterable[1:]:
        if item > tMax:
            tMax = item
        elif item < tMin:
            tMin = item
            
    return (tMax, tMin)


编写函数，模拟内置函数all()、any()和zip()。


In [147]:
def myAll(iterable):
    '''模拟内置函数all()'''
    #只要有一个元素等价于False，返回False
    for item in iterable:
        if not item:
            return False
    #如果所有元素都等价于True，返回True
    return True
def myAny(iterable):
    '''模拟内置函数any()'''
    #只要有一个元素等价于True，返回True
    for item in iterable:
        if item:
            return True
    #如果所有元素都等价于False，返回False
    return False
def myZip(*iterables):
    '''模拟内置函数zip()'''
    #获取所有迭代对象的最小长度
    min_length = min(map(len,iterables))
    
    #依次返回所有迭代对象中对应位置上元素组成的元组
    for i in range(min_length):
        yield tuple((it[i] for it in iterables))


编写函数，使用非递归算法实现冒泡排序算法。

In [148]:
from random import randint

def bubbleSort(lst, reverse=False):
    length = len(lst)
    for i in range(0, length):
        flag = False
        for j in range(0, length-i-1):
            #比较相邻两个元素大小，并根据需要进行交换，默认升序排序
            exp = 'lst[j] > lst[j+1]'
            #如果reverse=True则降序排序
            if reverse:
                exp = 'lst[j] < lst[j+1]'
            if eval(exp):
                lst[j], lst[j+1] = lst[j+1], lst[j]
                #flag=True表示本次扫描发生过元素交换
                flag = True
        #如果一次扫描结束后，没有发生过元素交换，说明已经按序排列
        if not flag:
            break


编写函数，使用递归算法实现冒泡排序算法。

In [149]:
from random import randint

def bubbleSort(lst, end=None, reverse=False):
    if end==None:
        length = len(lst)
    else:
        length = end
    if length<=1:
        return
    flag = False                          #flag用来标记本次扫描过程中是否发生了元素的交换
    for j in range(length-1):
        exp = 'lst[j] > lst[j+1]'         #比较相邻两个元素大小，并根据需要进行交换，默认升序排序
        if reverse:                       #如果reverse=True则降序排序
            exp = 'lst[j] < lst[j+1]'
        if eval(exp):
            lst[j], lst[j+1] = lst[j+1], lst[j]
            flag = True
    if flag==False:                       #如果没有发生元素交换，则表示已按序排列
        return
    else:
        bubbleSort(lst, length-1, reverse)#对剩余的元素进行排序


编写函数，模拟选择法排序。


In [150]:
def selectSort(lst, reverse=False):
    length = len(lst)
    for i in range(0, length):
        #假设剩余元素中第一个最小或最大
        m = i
        #扫描剩余元素
        for j in range(i+1, length):
            #如果有更小或更大的，就记录下它的位置
            exp = 'lst[j] < lst[m]'
            if reverse:
                exp = 'lst[j] > lst[m]'
            if eval(exp):
                m = j
        #如果发现更小或更大的，就交换值
        if m!=i:
            lst[i], lst[m] = lst[m], lst[i]


编写函数，模拟二分法查找。


In [151]:
def binarySearch(lst, value):
    start = 0
    end = len(lst)
    while start < end:
        #计算中间位置
        middle = (start + end) // 2
        #查找成功，返回元素对应的位置
        if value == lst[middle]:
            return middle
        #在后面一半元素中继续查找
        elif value > lst[middle]:
            start = middle + 1
        #在前面一半元素中继续查找
        elif value < lst[middle]:
            end = middle - 1
    #查找不成功，返回False
    return False


编写函数，模拟快速排序算法。


In [152]:
from random import randint

def quickSort(lst, reverse=False):
    if len(lst) <= 1:
        return lst
    #默认使用最后一个元素作为枢点
    pivot = lst.pop()
    first, second = [], []
    #默认使用升序排序
    exp = 'x<=pivot'
    #reverse=True表示降序排列
    if reverse == True:
        exp = 'x>=pivot'
    for x in lst:
        first.append(x) if eval(exp) else second.append(x)
    #递归调用
    return quickSort(first, reverse) + [pivot] + quickSort(second, reverse)
